### The following cells presents an execution tree enhanced with provenance.
### In this view, the wrong Evaluation is asked, and the provenance DAG is built based on the slice of the wrong evaluation. 

In [2]:
# Imports
import sqlite3

from graphviz import Graph

from debugprov.validity import Validity
from debugprov.node import Node
from debugprov.execution_tree import ExecutionTree
from debugprov.execution_tree_creator import ExecTreeCreator
from debugprov.top_down import TopDown
from debugprov.heaviest_first import HeaviestFirst
from debugprov.visualization import Visualization
from debugprov.provenance_enhancement import ProvenanceEnhancement 
from debugprov.single_stepping import SingleStepping
from debugprov.divide_and_query import DivideAndQuery

In [2]:
class CustomVisualization(Visualization):

    # CUSTOM ATTRIBUTES
    PROVENANCE_NODE_COLOR = 'lightblue'
    
    def generate_exec_tree(self, graph_name = 'exec_tree'):
        file_name = "{}.gv".format(graph_name)
        self.graph = Graph(graph_name, filename=file_name)
        self.graph.attr('node', shape='box')
        self.graph.attr('graph', ordering='out')
        root_node = self.exec_tree.root_node
        self.graph.node(str(root_node.ev_id), root_node.get_name(), fillcolor=self.INVALID_COLOR, style='filled') # root node
        self.navigate(root_node)
        eval_node = self.exec_tree.node_under_evaluation
        if eval_node is not None:
            self.graph.node(str(eval_node.ev_id), str(eval_node.get_name()), fillcolor=self.NODE_IN_EVALUATION, style='filled')
        buggy_node = self.exec_tree.buggy_node
        if buggy_node is not None:
            self.graph.node(str(buggy_node.ev_id), str(buggy_node.get_name()), fillcolor=self.BUGGY_NODE_COLOR, style='filled')
        if self.exec_tree.dependencies is not None:
            count = 0
            for d in self.exec_tree.dependencies: # this loop draws the provenance links between nodes
                count += 1
                print(" # {} ".format(count),end='')
                self.graph.edge(str(d.dependent.ev_id), str(d.influencer.ev_id), None, color=self.PROVENANCE_EDGE_COLOR, dir='forward')
                ## BEGIN customization
                self.graph.node(str(d.dependent.ev_id), None, fillcolor=self.PROVENANCE_NODE_COLOR, style='filled')
                self.graph.node(str(d.influencer.ev_id), None, fillcolor=self.PROVENANCE_NODE_COLOR, style='filled')
                ## END customization
    
    

In [3]:
NOW2_SQLITE_PATH = 'C:/Users/linha/Desktop/ws/py-scripts-examples/paper_example/.noworkflow/db.sqlite'

In [4]:
CURSOR = sqlite3.connect(NOW2_SQLITE_PATH).cursor()

In [5]:
creator = ExecTreeCreator(CURSOR)
exec_tree = creator.create_exec_tree()

In [6]:
prov = ProvenanceEnhancement(exec_tree, CURSOR)

In [7]:
wrong_data = prov.ask_wrong_data()

Which evaluation id is not correct? 101


In [8]:
wrong_data

In [9]:
len(prov.dependencies)

89

In [10]:
for e in prov.dependencies:
    if e.ev_id == wrong_data.ev_id:
        wd = e
        print("FOUND")
wd.code_component_name

FOUND


'print("maximum: {}".format(maximum))'

In [11]:
prov.enhance(wrong_data.ev_id)

KeyError: <debugprov.evaluation.Evaluation object at 0x000001AD1E6F2A90>

In [ ]:
len(exec_tree.dependencies)

In [ ]:
vis = Visualization(prov.exec_tree)

In [ ]:
vis.view_exec_tree('exec_tree_p3')

In [15]:
######################################################################################################################

In [16]:
for e in exec_tree.dependencies[wd]:
    print(e.code_component_name)

a(10)


In [17]:
wd.ev_id

7

In [18]:
reach = prov.dependencies
for item in reach:
    k = item
    v = reach[item] 
    print("# {} ".format(k.code_component_name))
    for elem in v:
        print("===- {}".format(elem.code_component_name))

# num 
# 10 
# num 
# x 
# x 
# j 
# j 
# j-3 
# 3 
# c(x) 
# c(x)*4 
===- c(x)
# 4 
# b(num) 
===- c(x)
# num 
===- b(num)
# num 
===- b(num)
# num*2 
===- b(num)
# 2 
# a(10) 
===- b(num)
# print(a(10)) 
===- a(10)


In [19]:
len(reach)
for item in reach:
    print(item.code_component_name)
    print("    {}".format(reach[item]))

num
    set()
10
    set()
num
    set()
x
    set()
x
    set()
j
    set()
j
    set()
j-3
    set()
3
    set()
c(x)
    set()
c(x)*4
    {<debugprov.evaluation.Evaluation object at 0x000002045DE54908>}
4
    set()
b(num)
    {<debugprov.evaluation.Evaluation object at 0x000002045DE54908>}
num
    {<debugprov.evaluation.Evaluation object at 0x000002045DE54F28>}
num
    {<debugprov.evaluation.Evaluation object at 0x000002045DE54F28>}
num*2
    {<debugprov.evaluation.Evaluation object at 0x000002045DE54F28>}
2
    set()
a(10)
    {<debugprov.evaluation.Evaluation object at 0x000002045DE54F28>}
print(a(10))
    {<debugprov.evaluation.Evaluation object at 0x000002045DE4E7F0>}
